In [8]:
import json
import operator

In [25]:
def clean_sub(sub, index_of_ad, index_of_score, expected_length):
    seen = []
    uniqs = []
    # explicitly check ordering
    i = 100
    sub.sort(key=operator.itemgetter(index_of_score), reverse=True)
    for entry in sub:
        if entry[index_of_score] > i:
            print "ORDERING PROBLEM"
            return None
        i = entry[index_of_score]
        #else:
        #    print type(entry)
    # de-dupe
    for entry in sub:
        if entry[index_of_ad] not in seen:
            uniqs.append(entry)
            seen.append(entry[index_of_ad])
    # explicitly sort by score after de-duping
    uniqs.sort(key=operator.itemgetter(index_of_score), reverse=True)
    # explicitly check ordering
    i = 100
    for entry in uniqs:
        if entry[index_of_score] > i:
            print "ORDERING PROBLEM"
            return None
        i = entry[index_of_score]
    return uniqs

In [10]:
pool_depth = 100

In [18]:
gt_file =  '../data/post_hoc_annotations/post_hoc_point_fact_combined.json'
    
with open(gt_file, 'r') as f:
    gt = eval(f.read())

# Submission Distribution

In [14]:
submission_path = '../data/team_submissions/'
submission_paths_dict = {}
submission_paths_dict['Georgetown'] = {}
submission_paths_dict['Georgetown']['NYU'] = submission_path + 'Georgetown/DomainDiscovery/NYU_pointfact.json'
submission_paths_dict['Georgetown']['JPL'] = submission_path + ('Georgetown/DomainDiscovery/'
                                                           'Georgetown_Submission_2_8/JPL/JPL_Point_Fact_2_9.json')
submission_paths_dict['Georgetown']['HG'] = submission_path + ('Georgetown/DomainDiscovery/'
                                                          'Georgetown_Submission_2_8/HG/HG_Point_Fact_2_9.json')

submission_paths_dict['Uncharted'] = {}
submission_paths_dict['Uncharted']['NYU'] = submission_path + ('Uncharted/DomainDiscovery/uncharted_NYU_DD.json')
submission_paths_dict['Uncharted']['JPL'] = submission_path + ('Uncharted/DomainDiscovery/uncharted_JPL_DD.json')
submission_paths_dict['Uncharted']['HG'] = submission_path + ('Uncharted/DomainDiscovery/uncharted_HG_DD.json')


submission_paths_dict['ISI'] = {}
submission_paths_dict['ISI']['NYU'] = submission_path + ('ISI/DomainDiscovery/'
                               'isi-nyu-answers-dig-extractions/'
                               'properly_formatted_submissions/'
                               'formatted_post_point_fact'
                               '_parsed_fixed_all_answers.json')
submission_paths_dict['ISI']['JPL'] = submission_path + ('ISI/DomainDiscovery/'
                               'jpl_answers_isi/'
                               'properly_formatted_submissions/'
                               'formatted_PF'
                               '-queries-parsed_all_answers.json')
submission_paths_dict['ISI']['HG'] = submission_path + ('ISI/DomainDiscovery/'
                               'hg_all_asnwers/'
                               'properly_formatted_submissions/'
                               'formatted_post_point_fact'
                               '_parsed_fixed_all_answers.json')

In [40]:
submitted_answers = {}
submitted_answers['NYU'] = {}
submitted_answers['JPL'] = {}
submitted_answers['HG'] = {}

for dataset in submitted_answers.keys():
    for qid in gt.keys():
        submitted_answers[dataset][qid] = []

## Georgetown

In [41]:
id_pos = 1
score_pos = 2
ans_length = 3 
id_length = 64
clus_type = 'Point Fact'

for dataset in submitted_answers.keys():
    file_path = submission_paths_dict['Georgetown'][dataset]
    f = open(file_path, 'r')
    data = eval(f.read())

    for qid in gt.keys():
        submissions = []
        for entry in data:
            # We are only evaluating some of them
            if entry['id'] == qid:
                submissions = clean_sub(entry['answer'], id_pos, score_pos, ans_length)
        if len(submissions) == 0:
            print 'NO SUBMISSION FOR QUESTION{0}'.format(qid)
        # Reconfirming no dupes
        if len(submissions) != len(list(set(tuple(i) for i in submissions))):
            print "PROBLEM WITH DUPLICATED DOC IDS"
            break
        # Now that submissions are sorted, only take top N
        if len(submissions) > pool_depth:
            submissions = submissions[:pool_depth]
            print "POOL DEPTH REACHED"
        submitted_ads = [entry[id_pos] for entry in submissions]
        submitted_answers[dataset][qid].extend(submitted_ads)

print ' '
for dataset in submitted_answers.keys():
    print dataset
    total = 0
    for qid in submitted_answers[dataset].keys():
        total += len(submitted_answers[dataset][qid])
    print total

POOL DEPTH REACHED
POOL DEPTH REACHED
NO SUBMISSION FOR QUESTION1647
NO SUBMISSION FOR QUESTION363
POOL DEPTH REACHED
NO SUBMISSION FOR QUESTION1647
NO SUBMISSION FOR QUESTION363
POOL DEPTH REACHED
 
HG
526
JPL
647
NYU
598


## ISI

In [42]:
id_pos = 1
score_pos = 2
ans_length = 3 
id_length = 64
clus_type = 'Point Fact'

for dataset in submitted_answers.keys():
    file_path = submission_paths_dict['ISI'][dataset]
    f = open(file_path, 'r')
    data = eval(f.read())

    for qid in gt.keys():
        submissions = []
        for entry in data:
            # We are only evaluating some of them
            if entry['question_id'].split('-')[0] == qid:
                submissions = clean_sub(entry['answer'], id_pos, score_pos, ans_length)
        # Reconfirming no dupes
        if len(submissions) != len(list(set(tuple(i) for i in submissions))):
            print "PROBLEM WITH DUPLICATED DOC IDS"
            break
        # Now that submissions are sorted, only take top N
        if len(submissions) > pool_depth:
            submissions = submissions[:pool_depth]
            print "POOL DEPTH REACHED"
        submitted_ads = [entry[id_pos] for entry in submissions]
        submitted_answers[dataset][qid].extend(submitted_ads)

print ' '
for dataset in submitted_answers.keys():
    print dataset
    total = 0
    for qid in submitted_answers[dataset].keys():
        total += len(submitted_answers[dataset][qid])
    print total

 
HG
1959
JPL
2147
NYU
740


## Uncharted

In [43]:
id_pos = 1
score_pos = 2
ans_length = 3 
id_length = 64
clus_type = 'Point Fact'

for dataset in submitted_answers.keys():
    file_path = submission_paths_dict['Uncharted'][dataset]
    f = open(file_path, 'r')
    all_data = eval(f.read())
    data = []
    for entry in all_data:
        if entry['questionType'] == clus_type:
            data.append(entry)

    for qid in gt.keys():
        submissions = []
        for entry in data:
            # We are only evaluating some of them
            if entry['question_id'] == qid:
                submissions = clean_sub(entry['answers'], id_pos, score_pos, ans_length)
        # Reconfirming no dupes
        if len(submissions) != len(list(set(tuple(i) for i in submissions))):
            print "PROBLEM WITH DUPLICATED DOC IDS"
            break
        # Now that submissions are sorted, only take top N
        if len(submissions) > pool_depth:
            submissions = submissions[:pool_depth]
            print "POOL DEPTH REACHED"
        submitted_ads = [entry[id_pos] for entry in submissions]
        submitted_answers[dataset][qid].extend(submitted_ads)
        
print ' '
for dataset in submitted_answers.keys():
    print dataset
    total = 0
    for qid in submitted_answers[dataset].keys():
        total += len(submitted_answers[dataset][qid])
    print total

POOL DEPTH REACHED
POOL DEPTH REACHED
POOL DEPTH REACHED
POOL DEPTH REACHED
POOL DEPTH REACHED
POOL DEPTH REACHED
POOL DEPTH REACHED
POOL DEPTH REACHED
POOL DEPTH REACHED
POOL DEPTH REACHED
POOL DEPTH REACHED
POOL DEPTH REACHED
POOL DEPTH REACHED
POOL DEPTH REACHED
POOL DEPTH REACHED
POOL DEPTH REACHED
POOL DEPTH REACHED
POOL DEPTH REACHED
POOL DEPTH REACHED
POOL DEPTH REACHED
POOL DEPTH REACHED
POOL DEPTH REACHED
POOL DEPTH REACHED
POOL DEPTH REACHED
POOL DEPTH REACHED
POOL DEPTH REACHED
POOL DEPTH REACHED
POOL DEPTH REACHED
POOL DEPTH REACHED
POOL DEPTH REACHED
POOL DEPTH REACHED
POOL DEPTH REACHED
POOL DEPTH REACHED
POOL DEPTH REACHED
 
HG
3239
JPL
3468
NYU
1934


In [45]:
uniq_subs = {}
for dataset in submitted_answers.keys():
    uniq_subs[dataset] = {}
    # Here we'll take the "unique submissions PER QUESTION"
    # Therefore, if the same document is considerred twice, 
    # once each for two different questions
    # that is considered as two "unique submissions"
    # since each time, the document has a new chance of being found relevant
    for qid in submitted_answers[dataset].keys():
        temp = submitted_answers[dataset][qid]
        temp = list(set(temp))
        uniq_subs[dataset][qid] = len(temp)

In [47]:
for dataset in uniq_subs.keys():
    print ' '
    print dataset
    total = 0
    for qid in uniq_subs[dataset].keys():
        total += uniq_subs[dataset][qid]
    print total

 
NYU
1825
 
HG
3102
 
JPL
3442


In [50]:
len(submitted_answers['NYU']['799'])

122